In [ ]:
from data_processing.Datasets import get_maestro_dataset, collate_fn, get_nes_dataset
from torch.utils.data import DataLoader
import torch
from model.transformer import Transformer
from functools import partial
import muspy
import copy

# NOTE: THIS DOWNLOADS SOMETHING, NEEDED TO GENERATE MUSIC WITH MUSPY
muspy.download_musescore_soundfont()


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# data hyperparams
SEQ_LEN = 500
BATCH_SIZE = 32
EVAL_BATCH_SIZE = 10

# model hyperparams
NUM_TOKENS = 388
DIM_MODEL = 512
NUM_HEADS = 2
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
DROPOUT_P = 0.1

REPRESENTATION = "event"  # set to either 'event' or 'pitch'


In [ ]:
# load device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
train_data, test_data = get_maestro_dataset(
    "data/maestro", representation=REPRESENTATION
)


In [ ]:
# Build dataloaders
train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    collate_fn=partial(collate_fn, seq_len=SEQ_LEN, device=device),
    shuffle=False,
)

val_dataloader = DataLoader(
    dataset=test_data,
    batch_size=EVAL_BATCH_SIZE,
    collate_fn=partial(collate_fn, seq_len=SEQ_LEN, device=device),
    shuffle=False,
)


In [ ]:
# create model, optim, criterion
model = Transformer(
    num_tokens=NUM_TOKENS,
    dim_model=DIM_MODEL,
    num_heads=NUM_HEADS,
    num_encoder_layers=NUM_ENCODER_LAYERS,
    num_decoder_layers=NUM_DECODER_LAYERS,
    dropout_p=DROPOUT_P,
).to(device)


In [ ]:
# collect data from dataset for a sample for metrics
data = torch.tensor(test_data.__getitem__(1), dtype=torch.long)
data = data[: 500].numpy()

dataset_sample = muspy.from_representation(data, 'event')

dataset_sample.show_pianoroll()

## Prepare Primer

This cell loads a small sample from the validation dataset that is passed into the transformer to be used as a baseline for generating more music.


In [ ]:
# prepare primer
primer = torch.tensor(test_data.__getitem__(12), dtype=torch.long)

labels = copy.deepcopy(primer)

# best so far has been item 10, with prime r
primer = primer[50: 150].to(device)
print(primer.shape)
labels = labels[50: 150].to(device)
print(labels.shape)

dataset_primer = primer.clone().detach()


In [ ]:
# load saved model params
model.load_state_dict(torch.load("./results/transformer/model_params/model-event-may-11.pth", map_location=torch.device(device)))
model.to(device)

# set to test
model.eval()

# pass in primer
with torch.no_grad():
    data = model.generate(primer.to(device), device, labels.to(device), target_seq_length=SEQ_LEN, single_token=True)

    # decode the returned info
    data = data.to('cpu').clone().detach().numpy()

    music = muspy.from_event_representation(data)

    muspy.write_audio("./results/transformer/music/test-generation.wav", music, "wav")

    muspy.show_pianoroll(music)


## Visualize the music


In [ ]:
# run some test metrics
# pitch range
pitch_range_dataset_sample = muspy.pitch_range(dataset_sample)
pitch_range_generated = muspy.pitch_range(music)

# polyphony (number of avg pitches being played concurrently)
polyphony_dataset_sample = muspy.polyphony(dataset_sample)
polyphony_generated = muspy.polyphony(music)

# empty beat rate
empty_beat_dataset_sample = muspy.empty_beat_rate(dataset_sample)
empty_beat_generated = muspy.empty_beat_rate(music)

print("Pitch range")
print(f"Dataset sample: {pitch_range_dataset_sample}, generated sample: {pitch_range_generated}")

print("Polyphony")
print(f"Dataset sample: {polyphony_dataset_sample}, generated sample: {polyphony_generated}")

print("Empty Beat Rate")
print(f"Dataset sample: {empty_beat_dataset_sample}, generated sample: {empty_beat_generated}")